First step is to read, and preprocess the data.

In [ ]:
%tensorflow_version 1.x
import tensorflow

TensorFlow 1.x selected.


In [ ]:
#imports
import os
import time
from google.colab import drive
import numpy as np
import matplotlib as plt
from matplotlib import pyplot
import pandas as pd

# Import keras modules
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
import keras
from keras import models, layers, Input, regularizers
from keras.utils import np_utils
from keras.models import Sequential, Model
import matplotlib.pyplot as plt
from keras.layers import Dense
import random
random.seed(42)

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Cancer project 2020/clean_data')

In [ ]:
#Toggles
self_sup = True
incep = False
aug = False
file_name = "Exp8v2p1"

In [ ]:
start_time = time.clock()
#Load data
mass_train_data = np.load('train_tensors.npy', allow_pickle=True)
mass_test_data = np.load('test_tensors.npy', allow_pickle=True)

calc_train_data = np.load('calc_train_tensors.npy', allow_pickle=True)
calc_test_data = np.load('calc_test_tensors.npy', allow_pickle=True)

In [ ]:
import cv2

#For each array, go through and resize. write func that takes desired size.
def fit_images(height, width, array):
    num_img = len(array)
    for i in range(num_img):
        array[i] = cv2.resize(array[i], dsize = (width, height), interpolation = cv2.INTER_CUBIC)
    return array    

def grey_to_3(array):
    num_img = len(array)
    for i in range(num_img):
        array[i] = cv2.cvtColor(array[i],cv2.COLOR_GRAY2RGB)
    return array    

In [ ]:
#I will try Inception, so change H,W
if incep == True:
    H=299
    W=299
else:
    H=224
    W=224

mass_train_img = fit_images(H, W, mass_train_data)
mass_test_img = fit_images(H,W, mass_test_data)

calc_train_img = fit_images(H, W, calc_train_data)
calc_test_img = fit_images(H,W, calc_test_data)

In [ ]:
mass_train_img = grey_to_3(mass_train_img)
mass_test_img = grey_to_3(mass_test_img)

calc_train_img = grey_to_3(calc_train_img)
calc_test_img = grey_to_3(calc_test_img)

In [ ]:
mass_train_img = np.stack(mass_train_img, axis=0)
mass_test_img = np.stack(mass_test_img, axis=0)

calc_train_img = np.stack(calc_train_img, axis=0)
calc_test_img = np.stack(calc_test_img, axis=0)

In [ ]:
mass_train_cl = np.ones(len(mass_train_img))
mass_test_cl = np.ones(len(mass_test_img))
calc_train_cl = np.zeros(len(calc_train_img))
calc_test_cl = np.zeros(len(calc_test_img))

train_img = np.concatenate((mass_train_img, calc_train_img))
test_img = np.concatenate((mass_test_img, calc_test_img))

train_cl = np.concatenate((mass_train_cl, calc_train_cl))
test_cl = np.concatenate((mass_test_cl, calc_test_cl))

In [ ]:
train_img = train_img.astype('float64') / 65535
test_img = test_img.astype('float64') / 65535

y_train = np_utils.to_categorical(train_cl, 2)
y_test = np_utils.to_categorical(test_cl, 2)

In [ ]:
#combined, so shuffle

In [ ]:
from sklearn.utils import shuffle

train_img, y_train = shuffle(train_img, train_cl)
test_img, y_test = shuffle(test_img, test_cl)

In [ ]:
y_train = np_utils.to_categorical(y_train, 2)
y_test = np_utils.to_categorical(y_test, 2)

In [ ]:
train_img.shape

(2865, 224, 224, 3)

In [ ]:
#End of preprocessing

In [ ]:
if incep == True:
    base_model = keras.applications.InceptionV3(
        include_top = False,
        weights = "imagenet")
else:
    base_model = keras.applications.DenseNet169(
        include_top = False,
        weights = "imagenet")    

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


51879936/51877672 [==============================] - 1s 0us/step


Next step is to load the pre-trained model, add our own dense layer on top, and train the dense layer.

In [ ]:
if self_sup == True:
    cp = ModelCheckpoint(filepath= file_name + 'run1.hdf5', verbose=1, monitor='val_loss', save_best_only=True)

In [ ]:
#no fine tuning

x = base_model.output
x = GlobalAveragePooling2D()(x)

x = layers.BatchNormalization()(x)
x = layers.Dense(1024, activation = 'relu', kernel_regularizer = regularizers.l2(0.01),)(x)
x = layers.Dropout(0.5)(x)

predictions = Dense(2, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


if self_sup == True:
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=["accuracy"])

    hist1 = model.fit(x=train_img,y = y_train, epochs=30, validation_data=(test_img, y_test), callbacks=[cp], verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 2865 samples, validate on 704 samples
Epoch 1/30
2865/2865 [==============================] - 24s 9ms/step - loss: 10.7721 - accuracy: 0.7283 - val_loss: 9.0078 - val_accuracy: 0.5092

Epoch 00001: val_loss improved from inf to 9.00784, saving model to Exp8v2p1run1.hdf5
Epoch 2/30
2865/2865 [==============================] - 11s 4ms/step - loss: 7.2478 - accuracy: 0.8152 - val_loss: 6.1775 - val_accuracy: 0.5433

Epoch 00002: val_loss improved from 9.00784 to 6.17748, saving model to Exp8v2p1run1.hdf5
Epoch 3/30
2865/2865 [==============================] - 11s 4ms/step - loss: 4.9132 - accuracy: 0.8403 - val_loss: 4.4236 - val_accuracy: 0.5220

Epoch 00003: val_loss improved from 6.17748 to 4.42362, saving model to Exp8v2p1run1.hdf5
Epoch 4/30
2865/2865 [==============================] - 11s 4ms/step - loss: 3.4673 - accuracy: 0.8571 - val_loss: 3.3828 - val_accuracy: 0.5057

Epoch 0

In [ ]:
if self_sup == True:
    cp2 = ModelCheckpoint(filepath= file_name + 'run2.hdf5', verbose=1, monitor='val_loss', save_best_only=True)

In [ ]:
if self_sup == True:
    #Fine tuning

    from keras.models import load_model

    for layer in model.layers:
      layer.trainable = True

    from keras.optimizers import SGD
    model.compile(optimizer=SGD(lr=0.00001, momentum=0.9), loss='binary_crossentropy', metrics=["accuracy"])

    hist2 = model.fit(x=train_img,y=y_train, batch_size=4, epochs=30, validation_data=(test_img, y_test), callbacks=[cp2], verbose=1)

Train on 2865 samples, validate on 704 samples
Epoch 1/30
2865/2865 [==============================] - 125s 44ms/step - loss: 0.5548 - accuracy: 0.8147 - val_loss: 1.5277 - val_accuracy: 0.5128

Epoch 00001: val_loss improved from inf to 1.52769, saving model to Exp8v2p1run2.hdf5
Epoch 2/30
2865/2865 [==============================] - 97s 34ms/step - loss: 0.5459 - accuracy: 0.8227 - val_loss: 0.6312 - val_accuracy: 0.7763

Epoch 00002: val_loss improved from 1.52769 to 0.63122, saving model to Exp8v2p1run2.hdf5
Epoch 3/30
2865/2865 [==============================] - 95s 33ms/step - loss: 0.5142 - accuracy: 0.8366 - val_loss: 0.6880 - val_accuracy: 0.7528

Epoch 00003: val_loss did not improve from 0.63122
Epoch 4/30
2865/2865 [==============================] - 95s 33ms/step - loss: 0.5017 - accuracy: 0.8372 - val_loss: 0.5132 - val_accuracy: 0.8281

Epoch 00004: val_loss improved from 0.63122 to 0.51322, saving model to Exp8v2p1run2.hdf5
Epoch 5/30
2865/2865 [=========================

In [ ]:
if self_sup == True:
    #learning rate decay
    
    lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    cp3 = ModelCheckpoint(filepath= file_name + '_run3.hdf5', verbose=1, monitor='val_loss', save_best_only=True)

In [ ]:
if self_sup == True:
    model.compile(optimizer=SGD(lr=0.000001), loss='binary_crossentropy', metrics=["accuracy"])

    hist3 = model.fit(x=train_img,y=y_train, batch_size=4, epochs=30, validation_data=(test_img, y_test), callbacks=[cp3], verbose=1)

Train on 2865 samples, validate on 704 samples
Epoch 1/30
2865/2865 [==============================] - 124s 43ms/step - loss: 0.2900 - accuracy: 0.9321 - val_loss: 0.3530 - val_accuracy: 0.9048

Epoch 00001: val_loss improved from inf to 0.35299, saving model to Exp8v2p1_run3.hdf5
Epoch 2/30
2865/2865 [==============================] - 95s 33ms/step - loss: 0.2845 - accuracy: 0.9372 - val_loss: 0.3499 - val_accuracy: 0.9027

Epoch 00002: val_loss improved from 0.35299 to 0.34988, saving model to Exp8v2p1_run3.hdf5
Epoch 3/30
2865/2865 [==============================] - 94s 33ms/step - loss: 0.2890 - accuracy: 0.9312 - val_loss: 0.3492 - val_accuracy: 0.9070

Epoch 00003: val_loss improved from 0.34988 to 0.34918, saving model to Exp8v2p1_run3.hdf5
Epoch 4/30
2865/2865 [==============================] - 95s 33ms/step - loss: 0.2973 - accuracy: 0.9333 - val_loss: 0.3568 - val_accuracy: 0.8963

Epoch 00004: val_loss did not improve from 0.34918
Epoch 5/30
2865/2865 [======================

In [ ]:
#Try on benign vs malignant

In [ ]:
#code prep

mass_train_img = mass_train_img.astype('float64') / 65535
mass_test_img = mass_test_img.astype('float64') / 65535

In [ ]:
with open('train_classes.txt') as file:
    labels = file.readlines()

train_classes = [x.strip() for x in labels]   

with open('test_classes.txt') as file:
    labels = file.readlines()

test_classes = [x.strip() for x in labels]

train_cl = []
for i in range(len(train_classes)):
   if train_classes[i] == 'MALIGNANT':
       train_cl.append(1)
   else:
       train_cl.append(0)    

test_cl = []
for i in range(len(test_classes)):
   if test_classes[i] == 'MALIGNANT':
       test_cl.append(1)
   else:
       test_cl.append(0)   

train_cl = np.array(train_cl)
test_cl = np.array(test_cl)       

train_cl

array([0, 1, 1, ..., 0, 1, 0])

In [ ]:
y_train3 = np_utils.to_categorical(train_cl, 2)
y_test3 = np_utils.to_categorical(test_cl, 2)

In [ ]:
#shuffle

train_img, y_train = shuffle(mass_train_img, y_train3)
test_img, y_test = shuffle(mass_test_img, y_test3)

In [ ]:
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
cp4 = ModelCheckpoint(filepath= file_name + '_run4.hdf5', verbose=1, monitor='val_accuracy', save_best_only=True)

In [ ]:
#benign vs malignant

if self_sup == True:
    model = load_model(file_name + '_run3.hdf5')

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=["accuracy"])


hist4 = model.fit(x=train_img,y=y_train, epochs=30, validation_data=(test_img, y_test), callbacks=[cp4], verbose=1)


Train on 1319 samples, validate on 378 samples
Epoch 1/30
1319/1319 [==============================] - 53s 40ms/step - loss: 0.9576 - accuracy: 0.6528 - val_loss: 2.4304 - val_accuracy: 0.6111

Epoch 00001: val_accuracy improved from -inf to 0.61111, saving model to Exp8v2p1_run4.hdf5
Epoch 2/30
1319/1319 [==============================] - 15s 11ms/step - loss: 0.4330 - accuracy: 0.8658 - val_loss: 1.2583 - val_accuracy: 0.6177

Epoch 00002: val_accuracy improved from 0.61111 to 0.61772, saving model to Exp8v2p1_run4.hdf5
Epoch 3/30
1319/1319 [==============================] - 15s 11ms/step - loss: 0.3097 - accuracy: 0.9177 - val_loss: 1.1974 - val_accuracy: 0.5966

Epoch 00003: val_accuracy did not improve from 0.61772
Epoch 4/30
1319/1319 [==============================] - 15s 11ms/step - loss: 0.2209 - accuracy: 0.9617 - val_loss: 1.5036 - val_accuracy: 0.6759

Epoch 00004: val_accuracy improved from 0.61772 to 0.67593, saving model to Exp8v2p1_run4.hdf5
Epoch 5/30
1319/1319 [======

In [ ]:
cp5 = ModelCheckpoint(filepath= file_name + '_run5.hdf5', verbose=1, monitor='val_accuracy', save_best_only=True)

for layer in model.layers:
   layer.trainable = True

from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.00001, momentum=0.9), loss='binary_crossentropy', metrics=["accuracy"])

hist2 = model.fit(x=train_img,y=y_train, batch_size=4, epochs=30, validation_data=(test_img, y_test), callbacks=[cp5], verbose=1)

Train on 1319 samples, validate on 378 samples
Epoch 1/30
1319/1319 [==============================] - 81s 61ms/step - loss: 0.4665 - accuracy: 0.8946 - val_loss: 1.5121 - val_accuracy: 0.6693

Epoch 00001: val_accuracy improved from -inf to 0.66931, saving model to Exp8v2p1_run5.hdf5
Epoch 2/30
1319/1319 [==============================] - 47s 36ms/step - loss: 0.3796 - accuracy: 0.9079 - val_loss: 2.0423 - val_accuracy: 0.5542

Epoch 00002: val_accuracy did not improve from 0.66931
Epoch 3/30
1319/1319 [==============================] - 47s 36ms/step - loss: 0.3202 - accuracy: 0.9234 - val_loss: 1.3141 - val_accuracy: 0.7090

Epoch 00003: val_accuracy improved from 0.66931 to 0.70899, saving model to Exp8v2p1_run5.hdf5
Epoch 4/30
1319/1319 [==============================] - 48s 36ms/step - loss: 0.3390 - accuracy: 0.8988 - val_loss: 1.1626 - val_accuracy: 0.6931

Epoch 00004: val_accuracy did not improve from 0.70899
Epoch 5/30
1319/1319 [==============================] - 47s 36ms/ste

In [ ]:
total_time = time.clock() - start_time
print(total_time)

12716.718787


In [ ]:
from keras.models import load_model
model = load_model(file_name + '_run5.hdf5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score, accuracy_score

probs = model.predict(mass_test_img)
y_pred = np.argmax(probs, axis = 1)
y_true = np.argmax(y_test3, axis = 1)

confusion_matrix(y_true, y_pred)

array([[183,  48],
       [ 42, 105]])

In [ ]:
print(f1_score(y_true, y_pred))
print()
print(roc_auc_score(y_true, y_pred))
print()
print(accuracy_score(y_true, y_pred))

0.7000000000000001

0.7532467532467532

0.7619047619047619
